# Importing Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
from object_detection.utils import dataset_util
import xml.etree.ElementTree as ET
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import shutil
from PIL import Image
from io import BytesIO
from object_detection.utils import config_util, label_map_util, visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection import model_lib_v2
from tensorflow.keras import mixed_precision

C:\Users\ACER\.conda\envs\tfobjdetection\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\ACER\.conda\envs\tfobjdetection\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an iss

# Directory Paths

In [2]:
REPO_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/models/research"
PIPELINE_CONFIG = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config"
MODEL_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/training/ssd_mobilenet_v2_model"
EXPORT_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/exported_model/ssd_mobilenet_v2_saved/saved_model"
LABEL_MAP_PATH = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/label_map.pbtxt"
EVAL_DIR = os.path.join(MODEL_DIR, 'eval_logs')

# Environment Setup

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
mixed_precision.set_global_policy('float32')
sys.path.append(os.path.join(REPO_DIR, 'slim'))
sys.path.append(os.path.join(REPO_DIR, 'object_detection'))

os.environ['PYTHONPATH'] = os.pathsep.join([
    os.environ.get('PYTHONPATH', ''),
    REPO_DIR,
    os.path.join(REPO_DIR, 'slim')
])

# Dataset Preprocessing and Augmentation

### Data Directory 

In [5]:
base_dir = r'C:\Users\ACER\Jupyter_Notebook_Workplace\Object-detection-dataset'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
output_label_map_path = os.path.join(base_dir, 'label_map.pbtxt')

### Extracting Label Map

In [6]:
# Extract labels from XML files
def extract_labels(directory):
    labels = set()
    for filename in os.listdir(directory):
        if filename.endswith(".xml"):
            tree = ET.parse(os.path.join(directory, filename))
            root = tree.getroot()
            for obj in root.findall('object'):
                labels.add(obj.find('name').text.strip())
    return labels

# Extract and merge all labels
train_labels = extract_labels(train_dir)
valid_labels = extract_labels(valid_dir)
all_labels = sorted(train_labels.union(valid_labels))

# Create label_map dictionary
label_map_dict = {i + 1: name for i, name in enumerate(all_labels)}

# Print labels to console
print("Labels used in your annotations:")
for idx, name in label_map_dict.items():
    print(f"{idx}: {name}")

# Save to label_map.pbtxt
def save_label_map(label_map, filepath):
    with open(filepath, 'w') as f:
        for idx, name in label_map.items():
            f.write("item {\n")
            f.write(f"  id: {idx}\n")
            f.write(f"  name: '{name}'\n")
            f.write("}\n\n")

save_label_map(label_map_dict, output_label_map_path)

print(f"\nLabel map saved to: {output_label_map_path}")

Labels used in your annotations:
1: Tom Tom
2: bench
3: bin
4: bus
5: car
6: chair
7: cng
8: dog
9: door
10: glass partition
11: motorcycle
12: person
13: pillar
14: railing
15: rickshaw
16: shelf
17: stair
18: table
19: tempu
20: tom tom
21: tree
22: truck
23: umbrella
24: van

Label map saved to: C:\Users\ACER\Jupyter_Notebook_Workplace\Object-detection-dataset\label_map.pbtxt


### Data Augmentation

In [7]:
def augment_image(image, bbox):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image, bbox

### Apply and Save Augmented Images

In [8]:
aug_dir = os.path.join(base_dir, 'train_augmented')
os.makedirs(aug_dir, exist_ok=True)

for file in os.listdir(train_dir):
    if file.endswith('.jpg') or file.endswith('.png'):
        base_name = os.path.splitext(file)[0]
        img_path = os.path.join(train_dir, file)
        xml_path = os.path.join(train_dir, base_name + '.xml')

        if not os.path.exists(xml_path):
            continue  # Skip if annotation missing

        # Load and augment image
        image = tf.io.read_file(img_path)
        image = tf.image.decode_jpeg(image, channels = 3)
        image = tf.image.resize(image, (320, 320))
        image = tf.cast(image, tf.uint8)

        aug_image, _ = augment_image(image, None)
        aug_image = tf.image.encode_jpeg(tf.cast(aug_image, tf.uint8))

        # Save augmented image
        aug_img_name = base_name + '_aug.jpg'
        aug_img_path = os.path.join(aug_dir, aug_img_name)
        tf.io.write_file(aug_img_path, aug_image)

        # Copy original XML with new name
        aug_xml_name = base_name + '_aug.xml'
        shutil.copy(xml_path, os.path.join(aug_dir, aug_xml_name))

### Merge Original and Augmented Training Data

In [9]:
train_combined_dir = os.path.join(base_dir, 'train_combined')
os.makedirs(train_combined_dir, exist_ok = True)

for src_dir in [train_dir, aug_dir]:
    for f in os.listdir(src_dir):
        shutil.copy(os.path.join(src_dir, f), os.path.join(train_combined_dir, f))

### Convert Pascal VOC to TFRecord

In [10]:
train_combined_dir = os.path.join(base_dir, 'train_combined')

# Generate label map from XMLs
def extract_labels(directory):
    labels = set()
    for filename in os.listdir(directory):
        if filename.endswith(".xml"):
            tree = ET.parse(os.path.join(directory, filename))
            root = tree.getroot()
            for obj in root.findall('object'):
                labels.add(obj.find('name').text.strip())
    return labels

train_labels = extract_labels(train_combined_dir)
valid_labels = extract_labels(valid_dir)
all_labels = sorted(train_labels.union(valid_labels))

# Create label map dict: {label_name: id}
label_map_dict = {name: i+1 for i, name in enumerate(all_labels)}

# Save to label_map.pbtxt
def save_label_map(label_map, filepath):
    with open(filepath, 'w') as f:
        for name, idx in label_map.items():
            f.write("item {\n")
            f.write(f"  id: {idx}\n")
            f.write(f"  name: '{name}'\n")
            f.write("}\n\n")

save_label_map(label_map_dict, output_label_map_path)
print(f"Label map saved to: {output_label_map_path}")

# Convert XML to TFRecord
def create_tf_example(xml_path, label_map_dict):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    image_path = xml_path.replace('.xml', '.jpg')
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found for {xml_path}")


    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image_data = fid.read()

    filename = os.path.basename(image_path).encode('utf8')
    image_format = b'jpg'
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    xmins, xmaxs, ymins, ymaxs, classes_text, classes = [], [], [], [], [], []
    for obj in root.findall('object'):
        label = obj.find('name').text.strip()
        classes_text.append(label.encode('utf8'))
        classes.append(label_map_dict[label])
        bbox = obj.find('bndbox')
        xmins.append(float(bbox.find('xmin').text) / width)
        xmaxs.append(float(bbox.find('xmax').text) / width)
        ymins.append(float(bbox.find('ymin').text) / height)
        ymaxs.append(float(bbox.find('ymax').text) / height)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def generate_tfrecord(output_path, image_dir, label_map_dict):
    writer = tf.io.TFRecordWriter(output_path)
    for filename in os.listdir(image_dir):
        if filename.endswith('.xml'):
            xml_path = os.path.join(image_dir, filename)
            try:
                tf_example = create_tf_example(xml_path, label_map_dict)
                writer.write(tf_example.SerializeToString())
            except Exception as e:
                print(f"Skipping {xml_path} due to error: {e}")
    writer.close()
    print(f"TFRecord written to: {output_path}")

# Generate train.record and val.record
generate_tfrecord(os.path.join(base_dir, 'train.record'), train_combined_dir, label_map_dict)
generate_tfrecord(os.path.join(base_dir, 'val.record'), valid_dir, label_map_dict)

Label map saved to: C:\Users\ACER\Jupyter_Notebook_Workplace\Object-detection-dataset\label_map.pbtxt
TFRecord written to: C:\Users\ACER\Jupyter_Notebook_Workplace\Object-detection-dataset\train.record
TFRecord written to: C:\Users\ACER\Jupyter_Notebook_Workplace\Object-detection-dataset\val.record


# Training MobileNetv2-SSD Model

### Training Loop

In [12]:
print("Starting training...")

model_lib_v2.train_loop(
    pipeline_config_path=PIPELINE_CONFIG,
    model_dir=MODEL_DIR,
    checkpoint_every_n=1000,
    record_summaries=True,
    num_steps_per_iteration=1
)

Starting training...
INFO:tensorflow:Maybe overwriting train_steps: None


[07/14 22:06:23] tensorflow INFO: Maybe overwriting train_steps: None


INFO:tensorflow:Maybe overwriting use_bfloat16: False


[07/14 22:06:23] tensorflow INFO: Maybe overwriting use_bfloat16: False


INFO:tensorflow:Reading unweighted datasets: ['C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/train.record']


...

INFO:tensorflow:Step 30100 per-step time 0.614s


[07/15 03:30:36] tensorflow INFO: Step 30100 per-step time 0.614s


INFO:tensorflow:{'Loss/classification_loss': 0.88227916,
 'Loss/localization_loss': 0.5579567,
 'Loss/regularization_loss': 0.06846487,
 'Loss/total_loss': 1.5087007,
 'learning_rate': 0.029394578}


[07/15 03:30:36] tensorflow INFO: {'Loss/classification_loss': 0.88227916,
 'Loss/localization_loss': 0.5579567,
 'Loss/regularization_loss': 0.06846487,
 'Loss/total_loss': 1.5087007,
 'learning_rate': 0.029394578}


KeyboardInterrupt: 

### Run Evaluations Continuously

In [13]:
print("Starting evaluation...")
model_lib_v2.eval_continuously(
    pipeline_config_path=PIPELINE_CONFIG,
    model_dir=MODEL_DIR,
    checkpoint_dir=MODEL_DIR,
    eval_dir=EVAL_DIR,
    wait_interval=300,
    timeout=3600
)

Starting evaluation...


[07/15 03:31:49] tensorflow WARNING: Forced number of epochs for all eval validations to be 1.


INFO:tensorflow:Maybe overwriting eval_dir: C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/training/ssd_mobilenet_v2_model\eval_logs


[07/15 03:31:49] tensorflow INFO: Maybe overwriting eval_dir: C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/training/ssd_mobilenet_v2_model\eval_logs


INFO:tensorflow:Ignoring config override key: eval_dir


...

[07/15 03:34:15] tensorflow INFO: 	+ Loss/regularization_loss: 0.068610


INFO:tensorflow:	+ Loss/total_loss: 2.028097


[07/15 03:34:15] tensorflow INFO: 	+ Loss/total_loss: 2.028097


INFO:tensorflow:Exiting evaluation at step 30000


[07/15 03:34:16] tensorflow INFO: Exiting evaluation at step 30000


### Export Trained Model

In [14]:
import subprocess

print("Exporting model...")
subprocess.run([
    "python",
    os.path.join(REPO_DIR, "object_detection", "exporter_main_v2.py"),
    "--input_type=image_tensor",
    "--pipeline_config_path=" + PIPELINE_CONFIG,
    "--trained_checkpoint_dir=" + MODEL_DIR,
    "--output_directory=" + os.path.dirname(EXPORT_DIR)
], check=True)

Exporting model...


CompletedProcess(args=['python', 'C:/Users/ACER/Jupyter_Notebook_Workplace/models/research\\object_detection\\exporter_main_v2.py', '--input_type=image_tensor', '--pipeline_config_path=C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', '--trained_checkpoint_dir=C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/training/ssd_mobilenet_v2_model', '--output_directory=C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/exported_model/ssd_mobilenet_v2_saved'], returncode=0)

### Load Exported Saved Model

In [15]:
print("Loading saved model...")
detect_fn = tf.saved_model.load(EXPORT_DIR)
infer = detect_fn.signatures['serving_default']

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

Loading saved model...


### Image Inference Utilities

In [ ]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (w, h) = image.size
    return np.array(image.getdata()).reshape((h, w, 3)).astype(np.uint8)

def visualize_detections(image_np, boxes, classes, scores, category_index, min_score_thresh=0.5):
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=min_score_thresh
    )
    return image_np_with_detections

def run_inference(image_path):
    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
    detections = infer(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {k: v[0, :num_detections].numpy() for k, v in detections.items()}
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    result = visualize_detections(
        image_np,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index
    )
    plt.figure(figsize=(12, 16))
    plt.imshow(result)
    plt.axis('off')
    plt.show()

### TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/training/ssd_mobilenet_v2_model'

## Post-training TFLite FP16 Quantization

In [ ]:
SAVED_MODEL_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/exported_model/ssd_mobilenet_v2_saved/saved_model"
TFLITE_MODEL_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/tflite_model"
TFLITE_MODEL_PATH = os.path.join(TFLITE_MODEL_DIR, "ssd_mobilenet_v2_fp16_flex.tflite")

os.makedirs(TFLITE_MODEL_DIR, exist_ok=True)

# Converter setup
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Enable fallback to TensorFlow ops (Flex ops)
converter.allow_custom_ops = True

# Convert
tflite_model = converter.convert()

# Save to file
with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"FP16 Flex TFLite model saved at: {TFLITE_MODEL_PATH}")

# Model Inference

In [ ]:
import cv2
from object_detection.utils import label_map_util

# === Paths ===
VIDEO_PATH = "C:/Users/ACER/Downloads/WhatsApp Video 2025-07-05 at 18.49.47_6a8d5809.mp4"
MODEL_DIR = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/exported_model/ssd_mobilenet_v2_saved/saved_model"
LABEL_MAP_PATH = "C:/Users/ACER/Jupyter_Notebook_Workplace/Object-detection-dataset/label_map.pbtxt"

# === Load model and label map ===
print("Loading model...")
detect_fn = tf.saved_model.load(MODEL_DIR)
print("Model loaded.")

category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

# === Load video ===
cap = cv2.VideoCapture(VIDEO_PATH)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    input_tensor = tf.convert_to_tensor([cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)], dtype=tf.uint8)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {k: v[0, :num_detections].numpy() for k, v in detections.items()}
    boxes = detections['detection_boxes']
    scores = detections['detection_scores']
    classes = detections['detection_classes'].astype(np.int32)

    h, w, _ = frame.shape
    for i in range(num_detections):
        if scores[i] > 0.5:
            ymin, xmin, ymax, xmax = boxes[i]
            (left, right, top, bottom) = (xmin * w, xmax * w, ymin * h, ymax * h)
            cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)

            class_id = classes[i]
            label = category_index.get(class_id, {'name': f'Class {class_id}'}).get('name', 'N/A')
            cv2.putText(frame, f'{label}: {int(scores[i]*100)}%', (int(left), int(top)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)

    cv2.imshow("Object Detection - SSD MobileNet V2", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading model...
